In [1]:
import netCDF4 as nc
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Add
from tensorflow.keras.models import Model, load_model
import matplotlib.pyplot as plt
import matplotlib.colors

# tf.config.list_physical_devices()
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Load the data for both years
data = nc.Dataset('ERA5_Data/ERA5_21_to_22_data_small.nc')

In [3]:
u = np.moveaxis(data['u'], 1, 0)
v = np.moveaxis(data['v'], 1, 0)

# z = np.concatenate((data_2020['z'][:], data_2021['z'][:]), axis=0)
# u = np.concatenate((data_2020['u'][:], data_2021['u'][:]), axis=0)
# v = np.concatenate((data_2020['v'][:], data_2021['v'][:]), axis=0)

In [4]:
u[2].shape

(730, 13, 17)

In [5]:
# Preprocessing the data
# Normalize the data
scaler = MinMaxScaler()

u_925 = scaler.fit_transform(u[2].reshape(-1, 1)).reshape(u[2].shape)
v_925 = scaler.fit_transform(v[2].reshape(-1, 1)).reshape(v[2].shape)

In [6]:
u_925.shape

(730, 13, 17)

In [7]:
# Pad the data to get even dimensions
u_925 = np.pad(u_925, ((0, 0), (0, 1), (0, 1)), mode='constant')
v_925 = np.pad(v_925, ((0, 0), (0, 1), (0, 1)), mode='constant')

In [8]:
u_925.shape

(730, 14, 18)

In [9]:
# Combine the parameters to form a single dataset
data_combined = np.stack((u_925, v_925), axis=-1)

In [10]:
data_combined.shape

(730, 14, 18, 2)

In [11]:
# Split the data into training and validation sets
X_train_padded, X_val_padded = train_test_split(data_combined, test_size=0.3, shuffle=False)

In [12]:
X_train_padded.shape

(511, 14, 18, 2)

In [13]:
# Define ResNet block
def resnet_block(input_tensor, filters, kernel_size=(3, 3), strides=(1, 1)):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Adjust the shortcut connection
    if strides != (1, 1) or input_tensor.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(input_tensor)
    else:
        shortcut = input_tensor
    
    x = Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [14]:
# If we want to constrain the input to the model
# X_train_padded = np.split(X_train_padded, 5, axis=0)
# print(len(X_train_padded))

In [15]:
# Input requires us to provide the shape which is the H X W X C without the number of samples N.
print(X_train_padded[0].shape)
input = Input(shape=X_train_padded[0].shape)

(14, 18, 2)


In [16]:
# Encoder
x = resnet_block(input, 32)
x = resnet_block(x, 64)
x = resnet_block(x, 128)
x = resnet_block(x, 256)
encoded = resnet_block(x, 256, strides=(2, 2))

# Decoder
x = UpSampling2D((2, 2))(encoded)
x = resnet_block(x, 256)
x = UpSampling2D((2, 2))(encoded)
x = resnet_block(x, 128)
x = UpSampling2D((2, 2))(encoded)
x = resnet_block(x, 64)
x = UpSampling2D((1, 1))(x)
x = resnet_block(x, 32)
decoded = Conv2D(2, (3, 3), activation='sigmoid', padding='same')(x)

# Compile the autoencoder
autoencoder = Model(input, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [17]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 18, 2)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 18, 32)   608         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 14, 18, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 14, 18, 32)   0           ['batch_normalization[0][0]']

In [18]:
# Train the model 
history = autoencoder.fit(X_train_padded, X_train_padded, epochs=20, batch_size=32, validation_data=(X_val_padded, X_val_padded))

Epoch 1/20
16/16 [==============================] - 9s 190ms/step - loss: 0.0245 - accuracy: 0.7625 - val_loss: 0.0414 - val_accuracy: 0.6527
Epoch 2/20
16/16 [==============================] - 1s 79ms/step - loss: 0.0032 - accuracy: 0.8750 - val_loss: 0.0332 - val_accuracy: 0.6526
Epoch 3/20
16/16 [==============================] - 1s 78ms/step - loss: 0.0022 - accuracy: 0.8961 - val_loss: 0.0321 - val_accuracy: 0.6714
Epoch 4/20
16/16 [==============================] - 1s 78ms/step - loss: 0.0019 - accuracy: 0.8931 - val_loss: 0.0304 - val_accuracy: 0.6867
Epoch 5/20
16/16 [==============================] - 1s 78ms/step - loss: 0.0021 - accuracy: 0.8844 - val_loss: 0.0323 - val_accuracy: 0.6696
Epoch 6/20
16/16 [==============================] - 1s 77ms/step - loss: 0.0017 - accuracy: 0.8872 - val_loss: 0.0343 - val_accuracy: 0.7020
Epoch 7/20
16/16 [==============================] - 1s 78ms/step - loss: 0.0015 - accuracy: 0.8828 - val_loss: 0.0330 - val_accuracy: 0.6502
Epoch 8/20
1

In [19]:
# If we want to constrain the input to the model
# for i in range(1,5):
#     history = autoencoder.fit(X_train_padded[i], X_train_padded[i], epochs=20, batch_size=16, validation_data=(X_val_padded, X_val_padded))

In [20]:
# Plot loss per iteration
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# For increasing validation loss, it is a sign we are overfitting and can be reduced by normalization.

In [ ]:
# Save this model
autoencoder.save('Saved Models/ResNet_00Z_925_Autoencoder.keras')

In [ ]:
# Save the weights
autoencoder.save_weights('Saved Models/ResNet_00Z_925_Autoencoder_weights.h5')

In [ ]:
# Load the model
autoencoder = load_model('Saved Models/ResNet_00Z_925_Autoencoder.keras')

In [ ]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [21]:
# Define the encoder model. Find the low dimension layer according to the model summary. 'activation_15' should be the encoder layer
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('activation_9').output)

In [ ]:
# Save the history object 
import pandas as pd
import pickle

# Convert the history.history dict to a pandas DataFrame
hist_df = pd.DataFrame(history.history)

# Save to csv
hist_csv_file = 'Saved Models/history_925.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# Save to pickle
with open('Saved Models/history_925.pkl', 'wb') as file:
    pickle.dump(history.history, file)


In [ ]:
# Load the saved history 
loaded_history = pd.read_csv('Saved Models/history_925.csv')

with open('Saved Models/history_925.pkl', 'rb') as file:
    loaded_history = pickle.load(file)

In [22]:
# Generate Encoded Representations for the Database
encoded_database = encoder.predict(data_combined)

23/23 [==============================] - 1s 28ms/step


In [23]:
print(encoded_database.shape)

(730, 7, 9, 256)


In [24]:
# Provide a test image for similarity check
# Load the data for input say
input_data = nc.Dataset('ERA5_Data/ERA5_28Nov23Input.nc')

# Extract data
u_input = np.moveaxis(input_data['u'], 1, 0)
v_input = np.moveaxis(input_data['v'], 1, 0)

u_input.shape
# Input data [0][0] is 700hPa, [0][1] is 850hPa and [0][2] is 925hPa

(3, 1, 13, 17)

In [25]:
input_data['u'].shape

(1, 3, 13, 17)

In [26]:
# Normalize the data
scaler = MinMaxScaler()

u_input = scaler.fit_transform(u_input[2].reshape(-1, 1)).reshape(u_input[2].shape)
v_input = scaler.fit_transform(v_input[2].reshape(-1, 1)).reshape(v_input[2].shape)

# Pad the data to get even dimensions
u_input = np.pad(u_input, ((0, 0), (0, 1), (0, 1)), mode='constant')
v_input = np.pad(v_input, ((0, 0), (0, 1), (0, 1)), mode='constant')

In [27]:
print(u_input.shape)

(1, 14, 18)


In [28]:
# Combine the parameters to form a single dataset
input_data_combined = np.stack((u_input, v_input), axis=-1)

In [29]:
# Make sure shapes match between input_data and database data
print(input_data_combined.shape,data_combined.shape)

(1, 14, 18, 2) (730, 14, 18, 2)


In [30]:
# Visualize test image
# x, y = np.meshgrid(np.linspace(95, 120, 101), np.linspace(-5, 15, 81))
x = np.linspace(102, 106, 17)
y = np.linspace(3, 0, 13)

# Check lengths are consistent
print(len(x),len(y))
print(len(input_data['z'][0][2][0]), len(input_data['z'][0][2]))

17 13
17 13


In [31]:
def vector_to_rgb(angle, absolute):
    global max_abs

    # normalize angle
    angle = angle % (2 * np.pi)
    if angle < 0:
        angle += 2 * np.pi

    return matplotlib.colors.hsv_to_rgb((angle / 2 / np.pi, 
                                         absolute / max_abs, 
                                         absolute / max_abs))

In [32]:
# Visualize test image
# Plot 925 winds
# plt.quiver(x, y, input_data['u'][0][2], input_data['v'][0][2], scale_units='xy')

# plt.quiver(x, y, input_data['u'][0][2], input_data['v'][0][2], np.arctan2(input_data['v'][0][2], input_data['u'][0][2]),
#            angles='xy',
#            scale_units='xy', 
#            pivot='mid',
#            color='g'
#           )


X = x
Y = y
U = input_data['u'][0][2]
V = input_data['v'][0][2]

angles = np.arctan2(input_data['v'][0][2], input_data['u'][0][2])
lengths = np.sqrt(np.square(input_data['u'][0][2]) + np.square(input_data['v'][0][2]))

angles = np.arctan2(V, U)
lengths = np.sqrt(np.square(U) + np.square(V))

max_abs = np.max(lengths)
c = np.array(list(map(vector_to_rgb, angles.flatten(), lengths.flatten())))

%matplotlib qt
plt.quiver(X, Y, U, V, color=c)
plt.title('28 Nov 2023 925 winds')
plt.show()

# Visualisation looks correct compared iwht EC Deterministic image on Optic

In [33]:
input_day_encoded = encoder.predict(input_data_combined)

1/1 [==============================] - 0s 297ms/step


In [34]:
print("Input image encoded shape", input_day_encoded.shape)
print("Encoded database shape", encoded_database.shape)

Input image encoded shape (1, 7, 9, 256)
Encoded database shape (730, 7, 9, 256)


In [35]:
# Reshaping to allow calculating of euclidean distances
input_day_encoded_flat = input_day_encoded.reshape(1,-1)
encoded_database_flat = encoded_database.reshape(encoded_database.shape[0], -1)

print("Input image encoded flat shape:", input_day_encoded_flat.shape)
print("Encoded flat shape:", encoded_database_flat.shape)

Input image encoded flat shape: (1, 16128)
Encoded flat shape: (730, 16128)


In [36]:
# Calculate similarities and find the most similar day:
# Use <Euclidean distances> to calculate the Euclidean distances between the input day's encoded representation and the encoded representations of all the days in the database.

from sklearn.metrics.pairwise import euclidean_distances

# Calculate Euclidean distances 
distances = euclidean_distances(input_day_encoded_flat, encoded_database_flat)

# Find the index of the most similar day (smallest distance)
most_similar_day_index = np.argmin(distances)

# Retrieve the data of the most similar day
most_similar_day_data = encoded_database_flat[most_similar_day_index]

print(most_similar_day_index)
print(most_similar_day_data)

420
[0.19937384 0.5899825  0.5648222  ... 0.         0.         0.36583006]


In [40]:
# Visualise most similar database image
X = x
Y = y
U = data['u'][most_similar_day_index][2]
V = data['v'][most_similar_day_index][2]

angles = np.arctan2(V, U)
lengths = np.sqrt(np.square(U) + np.square(V))

max_abs = np.max(lengths)
c = np.array(list(map(vector_to_rgb, angles.flatten(), lengths.flatten())))

%matplotlib qt
plt.quiver(X, Y, U, V, color=c)
plt.title('Most similar day 925 winds')
plt.savefig("Visualisation/925_only_most_similar_wings.png")